# Preprocessing

In [1]:
import numpy as np
import seaborn as sns
from impyute.imputation.cs import mice
import pandas as pd ; pd.options.display.max_rows = 150
import matplotlib as mpl ; mpl.rcParams['axes.unicode_minus'] = False
import matplotlib.pyplot as plt ; plt.rcParams['font.family'] = 'AppleGothic'
import variable as var

## 공공데이터 출처
**myhome_xyfill**: https://www.myhome.go.kr/hws/portal/sch/selectRentalHouseInfoListView.do  
크롤링코드: https://github.com/statkwon/DACON_LH/blob/master/crawling/MyHome.ipynb  
마이홈포털의 데이터를 이용해 현재 보유하고 있는 데이터에 단지명과 주소 변수를 추가하였습니다.

**code_age_gender**: https://www.data.go.kr/data/15059813/fileData.do  
주소 변수를 활용하여 단지코드별로 실제 연령/성별대 입주민수 및 총입주민수를 계산했습니다.

아래는 공공데이터를 활용하기 위한 코드입니다.

In [2]:
code_age_gender = pd.read_csv('한국토지주택공사_임대주택 단지별 연령대별 성별정보_20210511.csv', encoding='CP949')
myhome_xyfill = pd.read_csv('myhome_xyfill.csv').drop_duplicates()
age_gender_info = pd.read_csv('age_gender_info.csv')

In [3]:
# Train/test data 불러오기 및 오류 데이터 제거
train = pd.read_csv('train.csv')
train_error = ['C2085', 'C1397', 'C2431', 'C1649', 'C1036', 'C1095', 'C2051', 'C1218', 'C1894', 'C2483', 'C1502', 'C1988']
train = train[~train['단지코드'].isin(train_error)].reset_index(drop=True)
test = pd.read_csv('test.csv')
test_error = ['C2675', 'C2335', 'C1327']
test = test[~test['단지코드'].isin(test_error)].reset_index(drop=True)
print(f'Train Set: {train.shape} | Test Set: {test.shape}')

Train Set: (2869, 15) | Test Set: (1008, 14)


In [4]:
# '임대보증금', '임대료'의 '-'를 0으로 변환
train['임대보증금'] = train['임대보증금'].replace('-', 0).astype('float64')
train['임대료'] = train['임대료'].replace('-', 0).astype('float64')
test['임대보증금'] = test['임대보증금'].replace('-', 0).astype('float64')
test['임대료'] = test['임대료'].replace('-', 0).astype('float64')
print(f'Train Set: {train.shape} | Test Set: {test.shape}')

Train Set: (2869, 15) | Test Set: (1008, 14)


In [5]:
# '임대보증금', '임대료' NA를 0으로 대체
train.loc[(train['임대건물구분'] == '상가') & (train['임대보증금'].isnull()), '임대보증금'] = 0
train.loc[(train['임대건물구분'] == '상가') & (train['임대료'].isnull()), '임대료'] = 0
test.loc[(test['임대건물구분'] == '상가') & (test['임대보증금'].isnull()), '임대보증금'] = 0
test.loc[(test['임대건물구분'] == '상가') & (test['임대료'].isnull()), '임대료'] = 0
print(f'Train Set: {train.shape} | Test Set: {test.shape}')

Train Set: (2869, 15) | Test Set: (1008, 14)


In [6]:
# Test Data의 '임대보증금', '임대료' NA 특정값으로 대체
test.loc[test['임대보증금'].isnull(), '임대보증금'] = [5787000.0, 5787000.0, 11574000.0]
test.loc[test['임대료'].isnull(), '임대료'] = [79980.0, 79980.0, 159960.0]
print(f'Train Set: {train.shape} | Test Set: {test.shape}')

Train Set: (2869, 15) | Test Set: (1008, 14)


In [7]:
# NA를 0으로 변환
train.replace(np.nan, 0, inplace=True)
test['도보 10분거리 내 지하철역 수(환승노선 수 반영)'] = test['도보 10분거리 내 지하철역 수(환승노선 수 반영)'].fillna(0)
print(f'Train Set: {train.shape} | Test Set: {test.shape}')

Train Set: (2869, 15) | Test Set: (1008, 14)


In [8]:
# '임대건물구분 == 상가' 데이터를 지우고 Train_age/test_age 생성(공공데이터 병합을 위해 생성)
train_age = train[train['임대건물구분'] != '상가'].reset_index(drop=True)
train_age.drop(['임대건물구분'], axis=1, inplace=True)

test_age = test[test['임대건물구분'] != '상가'].reset_index(drop=True)
test_age.drop(['임대건물구분'], axis=1, inplace=True)

In [9]:
# '전용면적별세대수합' 변수 생성
noh_by_area = train.groupby('단지코드', as_index=False).agg({'전용면적별세대수':'sum'}).rename(columns={'전용면적별세대수':'전용면적별세대수합'})
train = pd.merge(left=train, right=noh_by_area, how='left', on='단지코드')
noh_by_area = test.groupby('단지코드', as_index=False).agg({'전용면적별세대수':'sum'}).rename(columns={'전용면적별세대수':'전용면적별세대수합'})
test = pd.merge(left=test, right=noh_by_area, how='left', on='단지코드')
print(f'Train Set: {train.shape} | Test Set: {test.shape}')

# '전용면적별세대수합' 변수 생성(공공데이터병합용)
noh_by_area = train_age.groupby('단지코드', as_index=False).agg({'전용면적별세대수':'sum'}).rename(columns={'전용면적별세대수':'전용면적별세대수합'})
train_age = pd.merge(left=train_age, right=noh_by_area, how='left', on='단지코드')
noh_by_area = test_age.groupby('단지코드', as_index=False).agg({'전용면적별세대수':'sum'}).rename(columns={'전용면적별세대수':'전용면적별세대수합'})
test_age = pd.merge(left=test_age, right=noh_by_area, how='left', on='단지코드')
print(f'Train Set: {train.shape} | Test Set: {test.shape}')

## '전용면적별세대수합_myhome' 변수 생성
train_myhome = train[train['공급유형']!='공공분양']
noh_by_area = train_myhome.groupby('단지코드',as_index=False).agg({'전용면적별세대수':'sum'}).rename(columns={'전용면적별세대수':'전용면적별세대수합_myhome'})
train = pd.merge(left=train, right=noh_by_area, how='left', on='단지코드')

test_myhome = test[test['공급유형']!='공공분양']
noh_by_area = test_myhome.groupby('단지코드', as_index=False).agg({'전용면적별세대수':'sum'}).rename(columns={'전용면적별세대수':'전용면적별세대수합_myhome'})
test = pd.merge(left=test, right=noh_by_area, how='left', on='단지코드')
print(f'Train Set: {train.shape} | Test Set: {test.shape}')

## '전용면적별세대수합_myhome' 변수 생성(공공데이터병합용)
train_age_myhome = train_age[train_age['공급유형']!='공공분양']
noh_by_area = train_age_myhome.groupby('단지코드',as_index=False).agg({'전용면적별세대수':'sum'}).rename(columns={'전용면적별세대수':'전용면적별세대수합_myhome'})
train_age = pd.merge(left=train_age, right=noh_by_area, how='left', on='단지코드')

test_age_myhome = test_age[test_age['공급유형']!='공공분양']
noh_by_area = test_age_myhome.groupby('단지코드', as_index=False).agg({'전용면적별세대수':'sum'}).rename(columns={'전용면적별세대수':'전용면적별세대수합_myhome'})
test_age = pd.merge(left=test_age, right=noh_by_area, how='left', on='단지코드')
print(f'Train Set: {train.shape} | Test Set: {test.shape}')

Train Set: (2869, 16) | Test Set: (1008, 15)
Train Set: (2869, 16) | Test Set: (1008, 15)
Train Set: (2869, 17) | Test Set: (1008, 16)
Train Set: (2869, 17) | Test Set: (1008, 16)


## '차량보유입주민수' 변수 생성

**총입주민수**  
myhome_xyfill과 code_age_gender를 이용하여 단지코드별로 총입주민수를 계산하였습니다.

**차량보유입주민수**  
202106_202106_연령별인구현황_월간.csv : https://jumin.mois.go.kr/ageStatMonth.do  
2021년_06월_자동차_등록자료_통계.xlsx : http://stat.molit.go.kr/portal/cate/statFileView.do?hRsId=58&hFormId=5

위 두 자료에 근거하여 남성의 경우 3-60대에서, 여성의 경우 4-50대에서 일인당 차량 보유 비율이 상대적으로 높게 나타났습니다.  
이에 해당 연령/성별대의 입주민수만 더하여 '차량보유입주민수' 변수를 생성했습니다.


In [10]:
# '차량보유입주민수' 변수 생성
member1, member2 = var.code_name(train_age), var.code_name(test_age)
member1, member2 = var.total_member(member1), var.total_member(member2)

age_col = ['단지코드','10대미만(여자)', '10대미만(남자)', '10대(여자)', '10대(남자)',
       '20대(여자)', '20대(남자)', '30대(여자)', '30대(남자)', '40대(여자)', '40대(남자)',
       '50대(여자)', '50대(남자)', '60대(여자)', '60대(남자)', '70대(여자)', '70대(남자)',
       '80대(여자)', '80대(남자)', '90대(여자)', '90대(남자)', '100대(여자)', '100대(남자)',
       '총입주민수']

member = pd.concat([member1[age_col], member2[age_col]])
member = member.drop_duplicates()
member.iloc[:,1:-1]=member.iloc[:,1:-1].mul(member['총입주민수'],axis=0)
member['차량보유입주민수'] = np.sum(member.iloc[:, [8, 9, 10, 11, 12, 14]], axis=1)
member = member[['단지코드', '차량보유입주민수', '총입주민수']]
train = pd.merge(train, member, how='left')
test = pd.merge(test, member, how='left')
print(f'Train Set: {train.shape} | Test Set: {test.shape}')

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/duckdeok/Desktop/parking/final/variable.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  myhome['지역'] = myhome['주소'].str.split(' ').str[0]
/Users/duckdeok/Desktop/parking/final/variable.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats i

Train Set: (2869, 19) | Test Set: (1008, 18)


## '임대보증금', '임대료' 변수 재생성
각 아파트별 전용면적별세대수합 대비 전용면적별세대수로 아파트별 임대보증금/임대료를 가중평균하여 해당 변수를 재생산하였습니다.

In [11]:
# '임대보증금', '임대료' 변수 재생성 (전용면적별세대수 가중 평균)
train['임대보증금'] = round(train['임대보증금'] * train['전용면적별세대수'] / train['전용면적별세대수합'])
rental_fee = train.groupby('단지코드', as_index=False).agg({'임대보증금':'sum'})
train = pd.merge(train, rental_fee, how='left', on='단지코드').drop(['임대보증금_x'], axis=1).rename(columns={'임대보증금_y':'임대보증금'})
train['임대료'] = round(train['임대료'] * train['전용면적별세대수'] / train['전용면적별세대수합'])
rental_fee = train.groupby('단지코드', as_index=False).agg({'임대료':'sum'})
train = pd.merge(train, rental_fee, how='left', on='단지코드').drop(['임대료_x'], axis=1).rename(columns={'임대료_y':'임대료'})
test['임대보증금'] = round(test['임대보증금'] * test['전용면적별세대수'] / test['전용면적별세대수합'])
rental_fee = test.groupby('단지코드', as_index=False).agg({'임대보증금':'sum'})
test = pd.merge(test, rental_fee, how='left', on='단지코드').drop(['임대보증금_x'], axis=1).rename(columns={'임대보증금_y':'임대보증금'})
test['임대료'] = round(test['임대료'] * test['전용면적별세대수'] / test['전용면적별세대수합'])
rental_fee = test.groupby('단지코드', as_index=False).agg({'임대료':'sum'})
test = pd.merge(test, rental_fee, how='left', on='단지코드').drop(['임대료_x'], axis=1).rename(columns={'임대료_y':'임대료'})
print(f'Train Set: {train.shape} | Test Set: {test.shape}')

Train Set: (2869, 19) | Test Set: (1008, 18)


## '소형세대' 변수 생성
전용면적이 40제곱미터보다 작은 경우에는 소형세대로 분리하여 해당 단지 내 소형세대의 단지수를 비율로 나타냈습니다.

In [12]:
# '소형세대' 변수 생성
train['세대구분'] = ['소형세대' if i < 40 else '기타세대' for i in train['전용면적']]
household = train.groupby(['단지코드', '세대구분'],
            as_index=False).agg({'전용면적별세대수':'sum'}).pivot('단지코드','세대구분', '전용면적별세대수').fillna(0).reset_index().drop(['기타세대'], axis=1)
train = pd.merge(train, household, how='left').drop(['세대구분'], axis=1)
train['소형세대'] = train['소형세대'] / train['전용면적별세대수합']

test['세대구분'] = ['소형세대' if i < 40 else '기타세대' for i in test['전용면적']]
household = test.groupby(['단지코드', '세대구분'],
            as_index=False).agg({'전용면적별세대수':'sum'}).pivot('단지코드', '세대구분', '전용면적별세대수').fillna(0).reset_index().drop(['기타세대'], axis=1)
test = pd.merge(test, household, how='left').drop(['세대구분'], axis=1)
test['소형세대'] = test['소형세대'] / test['전용면적별세대수합']
print(f'Train Set: {train.shape} | Test Set: {test.shape}')

Train Set: (2869, 20) | Test Set: (1008, 19)


## '전용면적' 변수 재생성
각 아파트별 전용면적별세대수합 대비 전용면적별세대수로 아파트별 전용면적을 가중평균하여 단지별 전용면적을 계산했습니다.

In [13]:
# '전용면적' 변수 재생성 (전용면적별세대수 가중 평균)
train['전용면적'] = round(train['전용면적'] * train['전용면적별세대수'] / train['전용면적별세대수합'])
rental_fee = train.groupby('단지코드', as_index=False).agg({'전용면적':'sum'})
train = pd.merge(train, rental_fee, how='left', on='단지코드').drop(['전용면적_x'], axis=1).rename(columns={'전용면적_y':'전용면적'})
test['전용면적'] = round(test['전용면적'] * test['전용면적별세대수'] / test['전용면적별세대수합'])
rental_fee = test.groupby('단지코드', as_index=False).agg({'전용면적':'sum'})
test = pd.merge(test, rental_fee, how='left', on='단지코드').drop(['전용면적_x'], axis=1).rename(columns={'전용면적_y':'전용면적'})
print(f'Train Set: {train.shape} | Test Set: {test.shape}')

Train Set: (2869, 20) | Test Set: (1008, 19)


## '차량보유인구비율' 변수 생성

In [14]:
# '차량보유인구비율' 변수 생성
cars = pd.DataFrame(age_gender_info.iloc[:, [7, 8, 9, 10, 12, 14]].sum(axis=1), columns=['차량보유인구비율'])
cars['지역'] = age_gender_info['지역']
train = pd.merge(train, cars, how='left', on='지역')
test = pd.merge(test, cars, how='left', on='지역')
print(f'Train Set: {train.shape} | Test Set: {test.shape}')

Train Set: (2869, 21) | Test Set: (1008, 20)


## '공급유형' 변수 재생성

단지별로 데이터를 통합하기 위하여 해당 단지의 특정 공급유형 포함 여부(0/1)로 더미 인코딩을 진행하였습니다.

In [15]:
# '공공임대(10년)', '공공임대(50년)', '공공임대(분납)', '국민임대', '영구임대', '행복주택', '임대상가' 변수 생성
supply_type = train.groupby(['단지코드', '공급유형'], as_index=False).size()
type1 = [1 if '행복주택' in np.array(supply_type.loc[supply_type['단지코드'] == i, '공급유형']) else 0 for i in train['단지코드'].unique()]
type2 = [1 if '영구임대' in np.array(supply_type.loc[supply_type['단지코드'] == i, '공급유형']) else 0 for i in train['단지코드'].unique()]
type3 = [1 if '임대상가' in np.array(supply_type.loc[supply_type['단지코드'] == i, '공급유형']) else 0 for i in train['단지코드'].unique()]
type4 = [1 if '공공임대(10년)' in np.array(supply_type.loc[supply_type['단지코드'] == i, '공급유형']) else 0 for i in train['단지코드'].unique()]
type5 = [1 if '공공임대(50년)' in np.array(supply_type.loc[supply_type['단지코드'] == i, '공급유형']) else 0 for i in train['단지코드'].unique()]
type6 = [1 if '공공임대(분납)' in np.array(supply_type.loc[supply_type['단지코드'] == i, '공급유형']) else 0 for i in train['단지코드'].unique()]
type7 = [1 if '국민임대' in np.array(supply_type.loc[supply_type['단지코드'] == i, '공급유형']) else 0 for i in train['단지코드'].unique()]
supply_type = pd.DataFrame(np.column_stack([train['단지코드'].unique(), type1, type2, type3, type4, type5, type6, type7]),
            columns=['단지코드','행복주택', '영구임대', '임대상가', '공공임대(10년)', '공공임대(50년)', '공공임대(분납)', '국민임대'])
train = pd.merge(train, supply_type, how='left')
supply_type = test.groupby(['단지코드', '공급유형'], as_index=False).size()
type1 = [1 if '행복주택' in np.array(supply_type.loc[supply_type['단지코드'] == i, '공급유형']) else 0 for i in test['단지코드'].unique()]
type2 = [1 if '영구임대' in np.array(supply_type.loc[supply_type['단지코드'] == i, '공급유형']) else 0 for i in test['단지코드'].unique()]
type3 = [1 if '임대상가' in np.array(supply_type.loc[supply_type['단지코드'] == i, '공급유형']) else 0 for i in test['단지코드'].unique()]
type4 = [1 if '공공임대(10년)' in np.array(supply_type.loc[supply_type['단지코드'] == i, '공급유형']) else 0 for i in test['단지코드'].unique()]
type5 = [1 if '공공임대(50년)' in np.array(supply_type.loc[supply_type['단지코드'] == i, '공급유형']) else 0 for i in test['단지코드'].unique()]
type6 = [1 if '공공임대(분납)' in np.array(supply_type.loc[supply_type['단지코드'] == i, '공급유형']) else 0 for i in test['단지코드'].unique()]
type7 = [1 if '국민임대' in np.array(supply_type.loc[supply_type['단지코드'] == i, '공급유형']) else 0 for i in test['단지코드'].unique()]
supply_type = pd.DataFrame(np.column_stack([test['단지코드'].unique(), type1, type2, type3, type4, type5, type6, type7]),
            columns=['단지코드', '행복주택', '영구임대', '임대상가', '공공임대(10년)', '공공임대(50년)', '공공임대(분납)', '국민임대'])
test = pd.merge(test, supply_type, how='left')
print(f'Train Set: {train.shape} | Test Set: {test.shape}')

Train Set: (2869, 28) | Test Set: (1008, 27)


## 불필요한 변수 제거

공급유형과 자격유형에 대해 교차표를 만들었을 때, 공급유형이 대부분의 자격유형 데이터를 설명하고 있어 자격유형 변수를 제거하였습니다.

In [16]:
# 불필요한 변수 제거
train.drop(['임대건물구분', '공급유형', '전용면적별세대수', '자격유형', '전용면적별세대수합'], axis=1, inplace=True)
test.drop(['임대건물구분', '공급유형', '전용면적별세대수', '자격유형', '전용면적별세대수합'], axis=1, inplace=True)
print(f'Train Set: {train.shape} | Test Set: {test.shape}')

Train Set: (2869, 23) | Test Set: (1008, 22)


## 데이터 단지단위 변환
단지단위로 데이터를 통합하였습니다.

---

In [17]:
train_ = train.drop_duplicates().reset_index(drop=True)
test_ = test.drop_duplicates().reset_index(drop=True)
print(f'Train Set: {train_.shape} | Test Set: {test_.shape}')

Train Set: (411, 23) | Test Set: (147, 22)


## 최종사용변수
|종류|변수명|
|:---:|:----:|
|기존변수(변형)|단지내주차면수|
|기존변수(변형)|전용면적|
|기존변수(변형)|총세대수|
|기존변수(변형)|공가수|
|추가변수|소형세대|
|추가변수|차량보유인구비율|
|추가변수|차량보유입주민수|
|공급유형변수|행복주택|
|공급유형변수|영구임대|
|공급유형변수|임대상가|
|공급유형변수|공공임대(10년)|
|공급유형변수|국민임대|
  
  


In [18]:
X_train = train_[['단지내주차면수', '전용면적', '차량보유인구비율', '총세대수', '공가수', '소형세대', '차량보유입주민수',
                  '행복주택', '영구임대', '임대상가', '공공임대(10년)', '국민임대']]
y_train = train_['등록차량수']

# Modeling

In [19]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler

In [20]:
# 피쳐에 2차항 및 상호작용항 추가
poly = PolynomialFeatures(2, include_bias=False)
X_train = poly.fit_transform(X_train)
print(X_train.shape)

(411, 90)


In [21]:
# StandardScaler 적용
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)

---

In [22]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso

**Linear Regression**

In [23]:
lr = LinearRegression()
kfold = KFold(n_splits=10, shuffle=True, random_state=11)
score = cross_val_score(lr, X_train, y_train, cv=kfold, scoring='neg_mean_absolute_error')
print(f'평균: {-score.mean()} | 표준편차: {score.std()}')

평균: 92017312376.44437 | 표준편차: 236107957851.66116


**Ridge Regression**

In [24]:
best_alpha = []
for i in range(50):
    rg = Ridge(alpha=i)
    kfold = KFold(n_splits=10, shuffle=True, random_state=11)
    score = cross_val_score(rg, X_train, y_train, cv=kfold, scoring='neg_mean_absolute_error')
    best_alpha.append(-score.mean())
print(f'Best alpha is {np.argmin(best_alpha)}.')

Best alpha is 17.


In [25]:
rg = Ridge(alpha=np.argmin(best_alpha))
kfold = KFold(n_splits=10, shuffle=True, random_state=11)
score = cross_val_score(rg, X_train, y_train, cv=kfold, scoring='neg_mean_absolute_error')
print(f'평균: {-score.mean()} | 표준편차: {score.std()}')

평균: 106.63969241987543 | 표준편차: 10.512571328786798


**Lasso Regression**

In [26]:
best_alpha = []
for i in range(1, 10):
    ls = Lasso(alpha=i)
    kfold = KFold(n_splits=10, shuffle=True, random_state=11)
    score = cross_val_score(ls, X_train, y_train, cv=kfold, scoring='neg_mean_absolute_error')
    best_alpha.append(-score.mean())
print(f'Best alpha is {np.argmin(best_alpha)}.')

Best alpha is 2.


In [27]:
ls = Lasso(np.argmin(best_alpha))
kfold = KFold(n_splits=10, shuffle=True, random_state=11)
score = cross_val_score(ls, X_train, y_train, cv=kfold, scoring='neg_mean_absolute_error')

In [28]:
print(f'평균: {-score.mean()} | 표준편차: {score.std()}')

평균: 105.54305623701052 | 표준편차: 10.896002954365605


In [29]:
-score

array([ 98.09705968, 108.77315993, 108.41004912, 114.75440486,
       113.72097469, 109.76228991,  84.79259996, 122.31022195,
       104.455613  ,  90.35418926])

In [30]:
ls.fit(X_train, y_train)

Lasso(alpha=2)

---

In [31]:
X_test = test_[['단지내주차면수', '전용면적', '차량보유인구비율', '총세대수', '공가수', '소형세대', '차량보유입주민수',
                  '행복주택', '영구임대', '임대상가', '공공임대(10년)', '국민임대']]

In [32]:
poly = PolynomialFeatures(2, include_bias=False)
X_test = poly.fit_transform(X_test)

In [33]:
X_test = scaler.transform(X_test)

In [34]:
test_['등록차량수'] = ls.predict(X_test)

In [35]:
submission = test_[['단지코드', '등록차량수']].rename(columns={'단지코드':'code', '등록차량수':'num'})
error = pd.DataFrame(np.array([['C2675', 0], ['C2335', 0], ['C1327', 0]]), columns=['code', 'num'])
submission = pd.concat([submission, error]).reset_index(drop=True)
submission.to_csv('submission.csv', index=False)

In [36]:
submission

,code,num
0,C1072,750.756051
1,C1128,1227.407026
2,C1456,450.151102
3,C1840,484.580579
4,C1332,1202.382281
5,C1563,1769.010567
6,C1794,864.589862
7,C1640,474.566283
8,C1377,368.481917
9,C2072,310.876784


|종류|Test MAE|
|:--:|:--:|
|Public|105.4968492188|
|Private|105.5256734854|